## Multilayer network analysis

In [1]:
# import libraries
import os
import pandas as pd
import geopandas as gpd
import shapely
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import osmnx as ox

## Load the multilayer network and fix column types

Load using the `networkx` package, and fix the columns that are string to the appropriate type.

In [2]:
# set crs
crs = 'EPSG:6677'

G_multilayer = nx.read_graphml('data/tokyo/multilayer_edit.graphml')

In [3]:
# weight by time and distance
time_dict = {}

for edge in G_multilayer.edges(data = True, keys = True):
    u = edge[0]
    v = edge[1]
    k = edge[2]
    # get layer
    layer = edge[3]['layer']


    if 'time' not in edge[3].keys():
        length = 0
        # set speed
        speed = 0.1
        match layer:
            case 'rail':
                length = float(edge[3]['distance'])
                speed = 50
            case 'bus':
                length = float(edge[3]['distance'])
                speed = 20
            case 'street':
                length = float(edge[3]['length'])
                speed = 4.8
            case _:
                speed = 0.1
        
        # calculate time
        time = length / speed * 60 / 1000

        time_dict[(u, v, k)] = {'time': time, 'weight_distance': length}

nx.set_edge_attributes(G_multilayer, time_dict)

# convert time into floats not strings
for u, v, d in G_multilayer.edges(data = True):
    d['time'] = float(d['time'])


## Combine with the zones data

In [4]:
# extract zones data

# load the area data
zones = gpd.read_file(os.path.join('data', 'H30_gis', 'H30_kzone.shp'))
zones_tokyo = zones[zones['kzone'] < 700]

# create zone list
zones_dict = []
for idx, area in zones_tokyo.iterrows():
    node_id = 'zone_' + str(area['kzone'])
    c = area['geometry'].centroid
    y = c.y
    x = c.x
    zones_dict.append((node_id, {
        'y': y,
        'x': x,
        'zone_int': area['kzone'],
        'zone_name': node_id
    }))


In [5]:
# merge with data

# extract the street graph
street_graph = nx.subgraph(G_multilayer, [n[0] for n in G_multilayer.nodes(data = True) if n[1]['layer'] == 'street'])

# get the largest strongly connected component
street_graph = street_graph.subgraph(max(nx.strongly_connected_components(street_graph), key = len))

In [6]:
# initialise edge list
edge_dict_street = []

for z in zones_dict:
    id = z[0]
    data = z[1]
    nearest_node = ox.nearest_nodes(street_graph, data['x'], data['y'], return_dist = True)
    data_dict = {
        'length': nearest_node[1],
        'type': 'connection',
        'time': nearest_node[1] / 4 * 60 / 1000
    }
    edge_dict_street.append((id, nearest_node[0], data_dict))
    edge_dict_street.append((nearest_node[0], id, data_dict))
    
G_multilayer.add_nodes_from(zones_dict)
G_multilayer.add_edges_from(edge_dict_street)

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

In [7]:
# save network
nx.write_graphml(G_multilayer, 'data/tokyo/multimodal_network_updated_weight.graphml', encoding = 'utf-8')

In [8]:
nx.shortest_path(G_multilayer, 'zone_410', 'zone_511', weight = 'weight_distance')

['zone_410',
 'street_928159268',
 'street_928165123',
 'street_667802170',
 'street_6205227295',
 'street_667802219',
 'street_1761889061',
 'street_1064956512',
 'street_393095070',
 'street_1761889055',
 'street_1761889059',
 'street_6237521382',
 'street_1304563603',
 'street_1304562765',
 'street_1686024828',
 'street_1686024797',
 'street_1686024938',
 'street_1304565737',
 'street_1304561233',
 'street_6138238823',
 'street_6138238824',
 'street_1064956455',
 'street_1304565463',
 'street_1304594791',
 'street_254238322',
 'street_1304594807',
 'street_1679243416',
 'street_744347580',
 'street_664482277',
 'street_1403578561',
 'street_744217574',
 'street_664482278',
 'street_6138221161',
 'street_6138221141',
 'street_6138221155',
 'street_6138221139',
 'street_6138221149',
 'street_754724560',
 'street_299225263',
 'street_260561538',
 'street_664487441',
 'street_1132298035',
 'street_1195341273',
 'street_1195341306',
 'street_928460016',
 'street_823938513',
 'street_1105

In [9]:
# convert wkt to geometry
for u, v, d in G_multilayer.edges(data = True):
    if 'geometry' in d.keys():
        d['geometry'] = shapely.wkt.loads(d['geometry'])

for n, d in G_multilayer.nodes(data = True):
    if 'geometry' in d.keys():
        d['geometry'] = shapely.wkt.loads(d['geometry'])
